## Activity 2: Building a Robo-Advisor
----

Portfolio optimization is intertwined with robo-advisors. One simple modification that we have to make is the fact that there are constraints added to the optimization problem. We are going to use four stocks and constrain the posititions in the following way:

- We are going to only allow for long positions (no negative values)
- A maximum amount of 40% is allowed in each asset
- The sum of all weights has to be equal to 1


In [1]:
import pandas as pd
import numpy as np
import scipy.optimize as sco

#Load data from Excel file
closingPrices = pd.read_excel("data_BackTest.xlsx",sheet_name="Closing Price")
#Build a column of type datetime
closingPrices["Date"] = pd.to_datetime(closingPrices.date,format='%Y-%m-%d') #transform from string into an object of type datetime
#Create data set for the analysis
symbol = ["VZ","AAPL","GOOGL","JPM"] #symbols for the analysis
b_date = pd.to_datetime("2012-01-01",format='%Y-%m-%d') #begining period
e_date = pd.to_datetime("2019-11-30",format='%Y-%m-%d') #end period
columns = ["Date"]
columns.extend(symbol)
data_analysis = closingPrices.loc[(closingPrices.Date>=b_date) & (closingPrices.Date<=e_date) ,columns].copy()

ret_assets = data_analysis[symbol].pct_change()
#compute annualized values
retPrt = ret_assets.mean()*250
CovMtx = ret_assets.cov()*250
print(retPrt)
print(CovMtx)

VZ       0.062657
AAPL     0.216997
GOOGL    0.194599
JPM      0.189164
dtype: float64
             VZ      AAPL     GOOGL       JPM
VZ     0.026855  0.006110  0.007564  0.009821
AAPL   0.006110  0.063026  0.023904  0.017960
GOOGL  0.007564  0.023904  0.051626  0.018306
JPM    0.009821  0.017960  0.018306  0.044672


Next, we define the function that we want to maximize

In [2]:
def min_func_meanVar(X,ret,Omega,gamma):
    #This function computes the negative of the objective function
    expRet = np.dot(X.T,ret) #This is equivalent to: np.sum(X*ret)
    expVar = np.dot(X.T,np.dot(Omega,X))
    val_funct = expRet - (gamma/2)*expVar
    return -1*val_funct

We are going to compute the aversion risk for the investor (look at the file "Risk Tolerance Assessment.pdf" for examples of on how to compute a score). I'm going to assume that a client has a risk-aversion of 1.5.

In [3]:
g = 1.5

We can check that different portfolio weights give different values:

In [4]:
print(min_func_meanVar(np.array([0.25,0.25,0.25,0.25]),retPrt,CovMtx,g))
print(min_func_meanVar(np.array([0.5,0.25,0.25,0.25]),retPrt,CovMtx,g))
print(min_func_meanVar(np.array([0.25,0.5,0.25,0.25]),retPrt,CovMtx,g))

-0.14928360324250284
-0.15896871202806528
-0.1901724487081155


In [5]:

args = (retPrt, CovMtx,g)
num_assets = len(retPrt)
init_x = np.ones(num_assets)*(1/num_assets) #set equal weight as starting point
#bnds = ((0, None), (0, None),(0, None), (0, None)) #No short positions 
bnds = ((0, 0.4), (0, 0.4),(0, 0.4), (0, 0.4)) #No short positions and a limit of 40% in each asset
#bnds = tuple((0,0.4) for asset in range(num_assets)) #this code generates tuples in an efficient way
cons = ({'type': 'eq', 'fun': lambda x:  x.sum()-1})
result = sco.minimize(min_func_meanVar, init_x, args=args, method='SLSQP',bounds=bnds,constraints=cons)
print(result)
print('The optimal portolio for this client is: (%s,%5.2f),(%s,%5.2f),(%s,%5.2f),(%s,%5.2f)'\
      %(symbol[0],result.x[0],symbol[1],result.x[1],symbol[2],result.x[2],symbol[3],result.x[3]))


     fun: -0.17785894727724344
     jac: array([-0.05116463, -0.16035154, -0.14883382, -0.15001085])
 message: 'Optimization terminated successfully.'
    nfev: 30
     nit: 5
    njev: 5
  status: 0
 success: True
       x: array([1.96023753e-16, 4.00000000e-01, 2.99067346e-01, 3.00932654e-01])
The optimal portolio for this client is: (VZ, 0.00),(AAPL, 0.40),(GOOGL, 0.30),(JPM, 0.30)
